<a href="https://colab.research.google.com/github/AKVITA380/Deep-Learning-6-July-2024/blob/main/Regression_with_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Regression With Keras

In [48]:
from warnings import filterwarnings
filterwarnings('ignore')

Step 1 Read Data

In [49]:
import pandas as pd
df=pd.read_csv("https://raw.githubusercontent.com/utkarshg1/Neural-Net-Datasets/main/PowerPlant.csv")
df

,AT,V,AP,RH,PE
0,8.34,40.77,1010.84,90.01,480.48
1,23.64,58.49,1011.40,74.20,445.75
2,29.74,56.90,1007.15,41.91,438.76
3,19.07,49.69,1007.22,76.79,453.09
4,11.80,40.66,1017.13,97.20,464.43
...,...,...,...,...,...
9563,15.12,48.92,1011.80,72.93,462.59
9564,33.41,77.95,1010.30,59.72,432.90
9565,15.99,43.34,1014.20,78.66,465.96
9566,17.65,59.87,1018.58,94.65,450.93


Step 2 : Perform Basic Data Quality

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9568 entries, 0 to 9567
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AT      9568 non-null   float64
 1   V       9568 non-null   float64
 2   AP      9568 non-null   float64
 3   RH      9568 non-null   float64
 4   PE      9568 non-null   float64
dtypes: float64(5)
memory usage: 373.9 KB


In [51]:
df.isna().sum()

AT    0
V     0
AP    0
RH    0
PE    0
dtype: int64

In [52]:
df.duplicated().sum()

41

In [53]:
df.duplicated().sum()/len(df)

0.004285117056856187

In [54]:
df=df.drop_duplicates(keep='first')
df

,AT,V,AP,RH,PE
0,8.34,40.77,1010.84,90.01,480.48
1,23.64,58.49,1011.40,74.20,445.75
2,29.74,56.90,1007.15,41.91,438.76
3,19.07,49.69,1007.22,76.79,453.09
4,11.80,40.66,1017.13,97.20,464.43
...,...,...,...,...,...
9563,15.12,48.92,1011.80,72.93,462.59
9564,33.41,77.95,1010.30,59.72,432.90
9565,15.99,43.34,1014.20,78.66,465.96
9566,17.65,59.87,1018.58,94.65,450.93


Step 3 Separate X AND Y

In [55]:
X=df.drop(columns=['PE'])
Y=df[['PE']]

In [56]:
X.head()

,AT,V,AP,RH
0,8.34,40.77,1010.84,90.01
1,23.64,58.49,1011.40,74.20
2,29.74,56.90,1007.15,41.91
3,19.07,49.69,1007.22,76.79
4,11.80,40.66,1017.13,97.20


In [57]:
Y.head()

,PE
0,480.48
1,445.75
2,438.76
3,453.09
4,464.43


Step 4 Preprocessing X using Pipeline

In [58]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [59]:
num_pipe=Pipeline(steps=[('impute', SimpleImputer(strategy='mean')),('scale',StandardScaler())]).set_output(transform='pandas')

In [60]:
X_pre= num_pipe.fit_transform(X)

In [61]:
X_pre.head()

,AT,V,AP,RH
0,-1.520448,-1.066041,-0.403535,1.141599
1,0.534897,0.330813,-0.309262,0.059223
2,1.354348,0.205475,-1.024725,-2.151400
3,-0.079020,-0.362884,-1.012941,0.236538
4,-1.055645,-1.074713,0.655349,1.633837


Step 5 Train Test Split

In [62]:
from sklearn.model_selection import train_test_split
xtrain,ytrain,xtest,ytest = train_test_split(X_pre,Y,test_size=0.2,random_state=42)

In [63]:
xtrain.head()

,AT,V,AP,RH
5703,-0.152905,0.324507,0.089713,1.817999
7322,-1.497611,-1.153542,0.323712,0.908830
9105,-0.885038,-0.954892,0.293410,0.988245
6436,1.398679,1.223949,-0.479290,-1.082031
6790,-0.824586,-0.942279,0.796758,1.506499


In [64]:
ytrain.head()

,AT,V,AP,RH
1087,1.827212,1.120683,-0.125767,-1.414069
6319,-1.501641,-1.244984,0.044260,-0.467246
8052,-1.465370,-1.418408,0.399466,-1.082031
9524,0.150695,0.648495,0.731104,-0.002393
4463,-0.036032,1.128565,-0.361449,1.586599


In [65]:
xtest.head()

,PE
5703,446.68
7322,483.54
9105,461.71
6436,437.38
6790,463.71


In [66]:
ytest.head()

,PE
1087,429.38
6319,485.29
8052,480.40
9524,452.30
4463,446.47


In [67]:
xtest.shape

(7621, 1)

In [68]:
ytest.shape

(1906, 1)

Step 6 Build the neural network

In [69]:
from keras.models import Sequential
from keras.layers import Dense,Input

In [70]:
#Intialize the model
model = Sequential()
# add Input layer
model.add(Input(shape=(xtrain.shape[1],)))
# add Dense layer
model.add(Dense(16,activation='relu'))
# add Hidden layer
model.add(Dense(8,activation='relu'))
#add oputput layer
model.add(Dense(1, activation='linear'))


In [76]:
# compile the model
model.compile(loss='mse', metrics=['mae'],optimizer='adam')

In [79]:
model.fit(xtrain,ytrain,
          validation_data=(xtest,ytest),
          epochs=52)

ValueError: Data cardinality is ambiguous:
  x sizes: 7621
  y sizes: 1906
Make sure all arrays contain the same number of samples.